### RAG pipelines - Data Ingestion to Vector DB PineLine


In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# 최근 LangChain은 구조가 바뀌면서 text_splitter가 langchain-text-splitters라는 별도 패키지로 분리됨
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

c:\Users\jh\AI-Project-List\rag-youtube\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data/pdf")

Found 2 PDF files to process

Processing: 1706.03762v7.pdf
  ✓ Loaded 15 pages

Processing: 회의실예약안내.pdf
  ✓ Loaded 2 pages

Total documents loaded: 17


In [7]:
all_pdf_documents

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '..\\data\\pdf\\1706.03762v7.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1', 'source_file': '1706.03762v7.pdf', 'file_type': 'pdf'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversit

In [9]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    
    # ✅ RecursiveCharacterTextSplitter: 텍스트를 재귀적으로 쪼개는 도구
    #   우선순위에 따라 줄바꿈 → 공백 → 문자 단위로 분할 시도
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,               # 각 청크의 최대 길이
        chunk_overlap=chunk_overlap,      # 청크 간 중복 구간 길이
        length_function=len,                   # 길이 계산 함수 (문자 수 기준)
        separators=["\n\n", "\n", " ", ""]   # 분할 우선순위 (문단→줄→단어→글자)
    )

    # 실제 문서 리스트를 분할하여 청크 리스트 생성
    split_docs = text_splitter.split_documents(documents)

    # 결과 요약 출력
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # 예시 청크 1개 출력
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")  # 청크 앞부분만 표시
        print(f"Metadata: {split_docs[0].metadata}")              # 원문서의 메타데이터 표시
    
    return split_docs

In [10]:
chunks=split_documents(all_pdf_documents)
chunks

Split 17 documents into 54 chunks

Example chunk:
Content: Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
...
Metadata: {'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '..\\data\\pdf\\1706.03762v7.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1', 'source_file': '1706.03762v7.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '..\\data\\pdf\\1706.03762v7.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1', 'source_file': '1706.03762v7.pdf', 'file_type': 'pdf'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversit

### embedding And vectorStoreDB

In [11]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
from typing import List
import numpy as np
from sentence_transformers import SentenceTransformer

class EmbeddingManager:
    """SentenceTransformer를 이용해 문서 임베딩을 생성·관리하는 클래스"""

    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        EmbeddingManager 초기화 메서드

        Args:
            model_name (str): 사용할 HuggingFace SentenceTransformer 모델 이름
                              (기본값: "all-MiniLM-L6-v2"), (고성능 영어용: intfloat/e5-small-v2)
        """
        self.model_name = model_name  # 모델 이름 저장
        self.model = None             # 모델 객체 (초기에는 로드되지 않음)
        self._load_model()            # 생성 시점에 모델 자동 로드

    def _load_model(self):
        """SentenceTransformer 모델을 로드하는 내부 메서드"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            # HuggingFace에서 지정한 SentenceTransformer 모델 로드
            self.model = SentenceTransformer(self.model_name)

            # 모델의 임베딩 벡터 차원 정보 출력
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")

        except Exception as e:
            # 모델 로드 중 오류 발생 시 메시지 출력 및 예외 전달
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        입력된 텍스트 리스트에 대한 임베딩을 생성하는 메서드

        Args:
            texts (List[str]): 임베딩을 생성할 문자열 리스트

        Returns:
            np.ndarray: shape = (len(texts), embedding_dim)
                        각 문장의 벡터 임베딩 배열
        """
        # 모델이 아직 로드되지 않았다면 예외 발생
        if not self.model:
            raise ValueError("Model not loaded")

        print(f"Generating embeddings for {len(texts)} texts...")

        # 모델을 이용해 입력 텍스트 리스트를 임베딩 벡터로 변환
        # show_progress_bar=True 옵션으로 진행 상태 표시
        embeddings = self.model.encode(texts, show_progress_bar=True)

        # 생성된 임베딩의 차원(shape) 출력
        print(f"Generated embeddings with shape: {embeddings.shape}")

        return embeddings


# --- 사용 예시 ---
# EmbeddingManager 객체 생성 (모델 자동 로드)
embedding_manager = EmbeddingManager()

# 객체 자체를 출력하면 모델 정보가 간단히 표시됨
embedding_manager


Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


### VectorStore

In [24]:
import os
import uuid
import numpy as np
import chromadb
from typing import List, Any

class VectorStore:
    """ChromaDB 벡터 저장소(Vector Database)를 이용해 문서 임베딩을 관리하는 클래스"""

    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        VectorStore 초기화 메서드

        Args:
            collection_name (str): 벡터 DB 내에서 사용할 컬렉션 이름 (테이블 같은 개념)
            persist_directory (str): 벡터 데이터를 로컬에 영구 저장할 디렉토리 경로
        """
        self.collection_name = collection_name         # ChromaDB 컬렉션 이름
        self.persist_directory = persist_directory     # 영구 저장 경로
        self.client = None                             # ChromaDB 클라이언트 객체
        self.collection = None                         # 특정 컬렉션 객체
        self._initialize_store()                       # 인스턴스 생성 시 바로 초기화 수행

    def _initialize_store(self):
        """ChromaDB 클라이언트 및 컬렉션 초기화 (보호된 내부 메서드)"""
        try:
            # 데이터 저장 디렉토리 생성 (이미 존재해도 오류 없이 통과)
            os.makedirs(self.persist_directory, exist_ok=True)

            # ChromaDB의 영구 저장(Persistent) 클라이언트 생성
            # → 지정한 경로에 DB 파일 형태로 저장됨
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            # 지정한 이름의 컬렉션을 가져오거나 없으면 새로 생성
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}  # 컬렉션 설명
            )

            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")

        except Exception as e:
            # 초기화 중 오류 발생 시 예외 출력 및 전달
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        문서 및 그 임베딩을 벡터 저장소(ChromaDB)에 추가하는 메서드

        Args:
            documents (List[Any]): LangChain Document 객체 리스트
            embeddings (np.ndarray): 각 문서에 대응되는 임베딩 벡터
        """
        # 문서 개수와 임베딩 개수가 다르면 오류 발생
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")

        print(f"Adding {len(documents)} documents to vector store...")

        # ChromaDB에 저장할 데이터 구조 초기화
        ids = []             # 각 문서의 고유 ID
        metadatas = []       # 각 문서의 메타데이터
        documents_text = []  # 문서의 실제 텍스트 내용
        embeddings_list = [] # 임베딩 벡터 리스트

        # 각 문서와 임베딩을 순회하면서 DB에 넣을 형태로 변환
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # 1. 문서마다 고유한 ID 생성 (uuid + 인덱스)
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            # 2. 문서 메타데이터 구성
            metadata = dict(doc.metadata)  # LangChain 문서의 기본 메타데이터 복사
            metadata['doc_index'] = i      # 문서 순번 추가
            metadata['content_length'] = len(doc.page_content)  # 텍스트 길이 저장
            metadatas.append(metadata)

            # 3. 문서 본문 텍스트 저장
            documents_text.append(doc.page_content)

            # 4. 임베딩을 list 형태로 변환하여 저장
            embeddings_list.append(embedding.tolist())

        # 5. 변환된 데이터를 ChromaDB 컬렉션에 추가
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )

            print(f"✅ Successfully added {len(documents)} documents to vector store")
            print(f"📦 Total documents in collection: {self.collection.count()}")

        except Exception as e:
            # 추가 중 오류가 발생한 경우
            print(f"Error adding documents to vector store: {e}")
            raise


# --- 사용 예시 ---
# VectorStore 객체 생성 → 자동으로 ChromaDB 초기화 및 컬렉션 생성
vectorstore = VectorStore()

# vectorstore 객체 출력 시, 내부적으로 초기화된 정보 확인 가능
vectorstore


Vector store initialized. Collection: pdf_documents
Existing documents in collection: 54


Retrieving documents for query: 'Tell me about Multi-Head Attention'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 32.67it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


[{'id': 'doc_bad82caf_12',
  'content': '3.2 Attention\nAn attention function can be described as mapping a query and a set of key-value pairs to an output,\nwhere the query, keys, values, and output are all vectors. The output is computed as a weighted sum\n3',
  'metadata': {'creator': 'LaTeX with hyperref',
   'content_length': 216,
   'moddate': '2024-04-10T21:11:43+00:00',
   'file_type': 'pdf',
   'trapped': '/False',
   'page': 2,
   'creationdate': '2024-04-10T21:11:43+00:00',
   'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
   'page_label': '3',
   'subject': '',
   'doc_index': 12,
   'keywords': '',
   'source_file': '1706.03762v7.pdf',
   'total_pages': 15,
   'producer': 'pdfTeX-1.40.25',
   'source': '..\\data\\pdf\\1706.03762v7.pdf',
   'author': '',
   'title': ''},
  'similarity_score': 0.1362038254737854,
  'distance': 0.8637961745262146,
  'rank': 1},
 {'id': 'doc_a8442ae1_16',
  'content': 'output values

In [14]:
chunks

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '..\\data\\pdf\\1706.03762v7.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1', 'source_file': '1706.03762v7.pdf', 'file_type': 'pdf'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversit

In [15]:
### 1️⃣ 텍스트 추출
# 'chunks'는 여러 문서(Document) 조각으로 구성된 리스트예요.
# 각 chunk에는 'page_content'(본문)와 'metadata'(출처 등)가 들어 있음.
# 여기서는 임베딩 생성을 위해 텍스트 내용만 추출합니다.
texts = [doc.page_content for doc in chunks]

### 2️⃣ 임베딩(Embeddings) 생성
# 위에서 추출한 모든 텍스트를 SentenceTransformer 모델에 넣어서
# 의미 벡터(embedding)로 변환합니다.
# 즉, 각 문서를 숫자 벡터로 표현한 것.
embeddings = embedding_manager.generate_embeddings(texts)

### 3️⃣ 벡터 DB 저장 (ChromaDB)
# 생성한 임베딩과 해당 원본 문서(chunks)를 ChromaDB에 저장합니다.
# 이 단계에서 각 문서는 (텍스트 + 임베딩 + 메타데이터)로 함께 저장되어
# 이후 유사도 검색(Semantic Search) 시에 빠르게 찾아낼 수 있습니다.
vectorstore.add_documents(chunks, embeddings)

Generating embeddings for 54 texts...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches: 100%|██████████| 2/2 [00:01<00:00,  1.55it/s]

Generated embeddings with shape: (54, 384)
Adding 54 documents to vector store...
✅ Successfully added 54 documents to vector store
📦 Total documents in collection: 54


### Retriever Pipeline From VectorStore

In [1]:
from typing import List, Dict, Any

class RAGRetriever:
    """사용자의 검색 질의(query)에 맞는 문서를 벡터 저장소(Vector Store)에서 찾아주는 클래스"""

    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        RAGRetriever 초기화

        Args:
            vector_store (VectorStore): 문서 임베딩이 저장된 ChromaDB 벡터 스토어 객체
            embedding_manager (EmbeddingManager): 쿼리 임베딩 생성을 담당하는 매니저
        """
        self.vector_store = vector_store              # 벡터 DB 접근용
        self.embedding_manager = embedding_manager    # 쿼리 임베딩 생성용 모델

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        사용자의 검색 질의(query)에 대해 유사도가 높은 문서를 벡터 DB에서 검색하는 메서드

        Args:
            query (str): 검색 질의(예: "What is LangChain?")
            top_k (int): 상위 몇 개의 문서를 반환할지 (기본값 5)
            score_threshold (float): 최소 유사도 점수 기준 (기본값 0.0)

        Returns:
            List[Dict[str, Any]]: 검색된 문서, 메타데이터, 유사도 점수를 포함한 딕셔너리 리스트
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")

        # 1️⃣ 쿼리 문장을 임베딩으로 변환
        # 사용자가 입력한 문자열을 SentenceTransformer를 이용해 벡터화
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        # 2️⃣ 벡터 스토어에서 쿼리 벡터와 가장 가까운 문서 검색
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],  # 쿼리 벡터 입력
                n_results=top_k                               # 상위 N개의 결과 반환
            )

            # 3️⃣ 검색 결과를 담을 리스트
            retrieved_docs = []

            # 검색 결과가 존재할 경우
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]   # 실제 문서 내용
                metadatas = results['metadatas'][0]   # 문서의 메타데이터
                distances = results['distances'][0]   # 거리 값 (cosine distance)
                ids = results['ids'][0]               # 문서 ID 리스트

                # 각 문서별로 정보 정리
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # cosine distance → similarity score로 변환 (1 - distance)
                    similarity_score = 1 - distance

                    # score_threshold 이상인 결과만 필터링
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,  # 1.0에 가까울수록 유사
                            'distance': distance,                   # 0.0에 가까울수록 유사
                            'rank': i + 1                           # 랭킹 순서
                        })

                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")

            return retrieved_docs

        # 검색 중 오류 발생 시
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []


# --- 사용 예시 ---
# RAGRetriever 객체를 생성하여 VectorStore와 EmbeddingManager를 연결
rag_retriever = RAGRetriever(vectorstore, embedding_manager)


NameError: name 'VectorStore' is not defined

In [ ]:
rag_retriever

In [ ]:
rag_retriever.retrieve("What is attention?")

Retrieving documents for query: 'What is attention?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 37.09it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)


[{'id': 'doc_c12167f4_12',
  'content': '3.2 Attention\nAn attention function can be described as mapping a query and a set of key-value pairs to an output,\nwhere the query, keys, values, and output are all vectors. The output is computed as a weighted sum\n3',
  'metadata': {'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
   'keywords': '',
   'subject': '',
   'doc_index': 12,
   'source_file': '1706.03762v7.pdf',
   'page_label': '3',
   'file_type': 'pdf',
   'total_pages': 15,
   'creator': 'LaTeX with hyperref',
   'author': '',
   'page': 2,
   'moddate': '2024-04-10T21:11:43+00:00',
   'source': '..\\data\\pdf\\1706.03762v7.pdf',
   'creationdate': '2024-04-10T21:11:43+00:00',
   'title': '',
   'content_length': 216,
   'trapped': '/False',
   'producer': 'pdfTeX-1.40.25'},
  'similarity_score': 0.380986750125885,
  'distance': 0.619013249874115,
  'rank': 1},
 {'id': 'doc_d1031b62_49',
  'content': 'Attention Visua

In [ ]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

### Initialize the Groq LLM (set your GROQ_API_KEY in environment)
groq_api_key = os.getenv("GROQ_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")


# llm=ChatGroq(groq_api_key=groq_api_key,model_name="gemma2-9b-it",temperature=0.1,max_tokens=1024)

llm = ChatOpenAI(
    model="gpt-4o-mini",  # 또는 gpt-4o
    temperature=0,
    api_key=openai_api_key
)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query,retriever,llm,top_k=3):
    ## retriever the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answwer using GROQ LLM
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response=llm.invoke([prompt.format(context=context,query=query)])
    return response.content

In [ ]:
answer=rag_simple("What is burger? and answer in korean",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'What is burger? and answer in korean'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  5.03it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)
No relevant context found to answer the question.


### Enhanced RAG Pipeline Features

In [ ]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    🔹 고급형 RAG 파이프라인 함수
    -----------------------------------------------------
    ✅ 기능 요약:
    - 질의(query)에 대해 retriever로 유사 문서 검색
    - 검색된 문서(context)를 LLM에 전달해 답변 생성
    - 문서 출처(sources), 신뢰도(confidence score), 전체 context 반환 가능
    -----------------------------------------------------
    파라미터 설명:
    query : str              -> 사용자가 입력한 질문
    retriever : 객체          -> .retrieve() 메서드를 가진 문서 검색기
    llm : 객체                -> LangChain의 ChatOpenAI, ChatGroq 등 LLM 모델 객체
    top_k : int              -> 상위 몇 개의 문서를 검색할지 (기본 5개)
    min_score : float        -> 유사도 최소 임계값 (이보다 낮은 문서는 제외)
    return_context : bool    -> True면 전체 context 텍스트를 결과에 포함
    """

    # 1️⃣ 유사 문서 검색
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)

    # 검색 결과가 없으면 기본 응답 반환
    if not results:
        return {
            'answer': 'No relevant context found.',
            'sources': [],
            'confidence': 0.0,
            'context': ''
        }

    # 2️⃣ 검색된 문서에서 context(내용)와 sources(출처) 정보 구성
    context = "\n\n".join([doc['content'] for doc in results])  # 문서 내용들을 이어 붙임

    # 각 문서별 메타데이터 정리
    sources = [{
        # 원본 파일명이나 출처 표시
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        # 페이지 정보 (PDF나 문서의 경우)
        'page': doc['metadata'].get('page', 'unknown'),
        # 유사도 점수
        'score': doc['similarity_score'],
        # 미리보기용 텍스트 일부 (300자까지만)
        'preview': doc['content'][:300] + '...'
    } for doc in results]

    # 가장 높은 유사도 점수를 신뢰도로 사용
    confidence = max([doc['similarity_score'] for doc in results])

    # 3️⃣ LLM에게 전달할 프롬프트 생성
    prompt = f"""
    Use the following context to answer the question concisely.
    Context:
    {context}

    Question: {query}

    Answer:
    """

    # 4️⃣ LLM 호출 → 프롬프트를 기반으로 답변 생성
    #    (prompt 안의 {context}, {query}는 format()으로 대체 가능)
    response = llm.invoke([prompt.format(context=context, query=query)])

    # 5️⃣ 결과 패키징
    output = {
        'answer': response.content,   # 모델이 생성한 답변
        'sources': sources,           # 참고된 문서들 목록
        'confidence': confidence      # 신뢰도 점수
    }

    # 요청 시 전체 context까지 포함
    if return_context:
        output['context'] = context

    return output


# ✅ 사용 예시
# retriever로 검색하고, LLM으로 답변 생성 후 결과 출력
result = rag_advanced(
    "Hard Negative Mining Techniques",  # 사용자가 한 질문
    rag_retriever,                      # 문서 검색기
    llm,                                # 언어모델 (ChatOpenAI, ChatGroq 등)
    top_k=3,                            # 상위 3개 문서만 검색
    min_score=0.1,                      # 최소 유사도 0.1 이상만 포함
    return_context=True                 # context 미리보기 포함
)

# ✅ 결과 확인용 출력
print("Answer:", result['answer'])                          # 모델의 최종 답변
print("Sources:", result['sources'])                        # 어떤 문서들이 사용됐는지
print("Confidence:", result['confidence'])                  # 신뢰도 점수
print("Context Preview:", result['context'][:300])          # context 일부 미리보기


In [ ]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    """
    🔹 고급형 RAG 파이프라인 클래스
    -----------------------------------------------------
    ✅ 주요 기능
    - 문서 검색 (Retriever)
    - 답변 생성 (LLM)
    - 실시간 출력 시뮬레이션 (Streaming)
    - 출처 표기 (Citations)
    - 요약 (Summarization)
    - 질의/응답 히스토리 저장 (History)
    -----------------------------------------------------
    retriever : 벡터DB 검색기 (예: Chroma, FAISS 등)
    llm       : LLM 모델 (예: ChatOpenAI, ChatGroq 등)
    """

    def __init__(self, retriever, llm):
        # RAG에 필요한 검색기(retriever)와 언어모델(LLM)을 주입받음
        self.retriever = retriever
        self.llm = llm
        self.history = []  # 이전 질의와 답변 기록을 저장하는 리스트

    # ------------------------------
    # 🧠 핵심 메서드: query()
    # ------------------------------
    def query(
        self,
        question: str,
        top_k: int = 5,
        min_score: float = 0.2,
        stream: bool = False,
        summarize: bool = False
    ) -> Dict[str, Any]:
        """
        ✅ 사용자 질문(question)에 대해 RAG 검색 + LLM 답변 생성 수행
        - top_k : 검색할 문서 개수
        - min_score : 최소 유사도 점수
        - stream : True면 출력 중간 과정을 실시간처럼 출력
        - summarize : True면 생성된 답변을 2문장으로 요약
        """

        # 1️⃣ Retriever를 통해 유사 문서 검색
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)

        # 검색 결과가 없을 때의 기본 처리
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            # 2️⃣ 문서 본문(context)과 출처(sources) 구성
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'   # 내용 미리보기
            } for doc in results]

            # 3️⃣ 모델에 전달할 프롬프트 구성
            prompt = f"""Use the following context to answer the question concisely.
            Context:
            {context}

            Question: {question}

            Answer:"""

            # 4️⃣ Streaming 모드 (답변 생성 전 프롬프트를 한 줄씩 출력하는 시뮬레이션)
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):  # 80자씩 잘라서 출력
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)  # 0.05초 간격으로 출력
                print()

            # 5️⃣ LLM 호출 → 답변 생성
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # 6️⃣ 출처 정보(Citations) 추가
        # 예: [1] doc1.pdf (page 3)
        citations = [
            f"[{i+1}] {src['source']} (page {src['page']})"
            for i, src in enumerate(sources)
        ]
        # 답변 맨 아래에 출처 목록을 추가
        answer_with_citations = (
            answer + "\n\nCitations:\n" + "\n".join(citations)
            if citations else answer
        )

        # 7️⃣ 요약 기능 (Summarization)
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content  # 요약된 텍스트 저장

        # 8️⃣ 히스토리 기록 (이전 질의와 답변을 누적 저장)
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        # 9️⃣ 결과 반환 (딕셔너리 형태)
        return {
            'question': question,                 # 입력 질문
            'answer': answer_with_citations,      # 출처가 포함된 답변
            'sources': sources,                   # 참고된 문서 목록
            'summary': summary,                   # 요약문 (옵션)
            'history': self.history               # 누적된 질의 기록
        }


# ------------------------------
# ✅ 사용 예시
# ------------------------------
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)

# 스트리밍 출력 + 요약 기능 켜고 질의
result = adv_rag.query(
    "what is attention is all you need",
    top_k=3,
    min_score=0.1,
    stream=True,       # 답변 생성 시 프롬프트를 실시간처럼 보여줌
    summarize=True     # 답변을 2문장으로 요약
)

# 최종 결과 확인
print("\nFinal Answer:", result['answer'])     # 모델이 생성한 답변 + 출처
print("Summary:", result['summary'])           # 요약된 결과
print("History:", result['history'][-1])       # 가장 최근의 질의 기록
